In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [8]:
df = pd.read_csv('/Users/sanjayroberts1/Desktop/data_science_tools1/sudeste.csv')

In [9]:
df = df.set_index('mdct')
df.index = pd.to_datetime(df.index)

Interpolation for temperature 

In [10]:
df.temp.unique()

array([29.3, 29. , 27.4, 25.8, 25.4, 23.8, 22. , 19.7, 18.3, 22.9, 25. ,
        0. , 31. , 32.7, 33.7, 34.7, 34.3, 32.6, 29.9, 31.5, 30.3, 30.2,
       28.1, 26.6, 25.1, 23.7, 21.5, 21.2, 19.2, 25.7, 27.7, 29.5, 31.1,
       33.1, 33.9, 35.1, 36.4, 36.6, 35.7, 33. , 28. , 31.4, 31.6, 28.5,
       27. , 22.7, 25.6, 24.7, 27.1, 34.2, 35.2, 35.8, 36.8, 36. , 32.8,
       28.4, 27.8, 30.6, 28.3, 26.7, 23.3, 22.5, 21. , 22.6, 31.3, 34.9,
       26.4, 24.8, 24. , 26.8, 24.4, 23.2, 25.5, 28.2, 30.7, 32.2, 34.1,
       35.4, 35.5, 33.4, 24.2, 21.9, 26. , 24.5, 22.4, 21.8, 23. , 29.2,
       32.4, 25.9, 24.3, 27.9, 22.1, 20.9, 20.5, 19.8, 19.4, 27.2, 28.9,
       30.1, 34.8, 35. , 32.3, 26.1, 24.9, 21.1, 22.3, 25.2, 29.1, 33.8,
       34.4, 34.5, 33.5, 31.8, 29.8, 28.6, 26.2, 28.7, 32. , 32.9, 32.5,
       30. , 31.2, 34. , 35.3, 31.7, 32.1, 31.9, 33.3, 33.2, 29.4, 24.6,
       30.8, 35.6, 26.9, 28.8, 35.9, 27.5, 26.3, 26.5, 29.6, 36.1, 27.6,
       27.3, 29.7, 33.6, 30.5, 34.6, 36.7, 23.9, 30

- Separate data for each station
- Each station may have it's own issues for data cleanup, so we separate and apply a cleanup function for each

In [ ]:
individual_stations = {}
for each in df.wsid.unique():
    is_ = df['wsid'] == each
    k = str(each)
    individual_stations[k] = df[is_]

In [ ]:
individual_stations.keys()

In [5]:
df['temp'].replace(0, np.nan, inplace=True)

In [6]:
df.interpolate('time', inplace=True, limit_direction='both')

In [ ]:
rain_df = station_385_year[['mo','prcp']].fillna(0)

In [ ]:
total_rain = rain_df.groupby('mo').sum().reset_index()

In [ ]:
total_rain.plot( x='mo', y='prcp', kind='bar')

In [ ]:
average_rain = rain_df.groupby('mo').mean().reset_index()

In [ ]:
average_rain.plot( x='mo', y='prcp', kind='bar')